# One-vs-all Logistic Regression

In [206]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import time 
from sklearn import datasets 
from matplotlib import cm
import seaborn as sns
from scipy.optimize import minimize
import math

In [207]:
iris_dataset = datasets.load_iris()

In [208]:
columns = iris_dataset.feature_names
y = iris_dataset.target

In [209]:
iris_dataframe = iris_dataset.data

In [210]:
iris_dataframe = pd.DataFrame(iris_dataframe, columns = columns)
iris_dataframe.insert(0, 'Intercept', 1)
iris_dataframe.insert(5, 'Label', y)

In [211]:
iris_dataframe

,Intercept,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Label
0,1,5.1,3.5,1.4,0.2,0
1,1,4.9,3.0,1.4,0.2,0
2,1,4.7,3.2,1.3,0.2,0
3,1,4.6,3.1,1.5,0.2,0
4,1,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...,...
145,1,6.7,3.0,5.2,2.3,2
146,1,6.3,2.5,5.0,1.9,2
147,1,6.5,3.0,5.2,2.0,2
148,1,6.2,3.4,5.4,2.3,2


# One-vs-all Training strategy: dividing in the 3 classes

In [212]:
class_0 = iris_dataframe[iris_dataframe['Label'] == 0]
class_1 = iris_dataframe[iris_dataframe['Label'] == 1]
class_2 = iris_dataframe[iris_dataframe['Label'] == 2]

# First classifier (label 0) Training set setup

In [213]:
classifier_1_pos = class_0.copy()
classifier_1_neg = pd.concat([class_1.copy(), class_2.copy()], axis=0)

In [214]:
classifier_1_pos['Label'] = 1
classifier_1_neg['Label'] = 0

In [215]:
x_1 = pd.concat([classifier_1_pos,classifier_1_neg], axis=0).iloc[:,0:5]
y_1 = pd.concat([classifier_1_pos,classifier_1_neg], axis=0).iloc[:,[-1]]

# Second classifier (label 1) training set setup

In [216]:
classifier_2_pos = class_1.copy()
classifier_2_neg = pd.concat([class_0.copy(), class_2.copy()], axis=0)

In [217]:
classifier_2_pos['Label'] = 1
classifier_2_neg['Label'] = 0

In [218]:
x_2 = pd.concat([classifier_2_pos,classifier_2_neg], axis=0).iloc[:,0:5]
y_2 = pd.concat([classifier_2_pos,classifier_2_neg], axis=0).iloc[:,[-1]]

# Third classifier (label 2) training set setup

In [219]:
classifier_3_pos = class_2.copy()
classifier_3_neg = pd.concat([class_0.copy(), class_1.copy()], axis=0)

In [220]:
classifier_3_pos['Label'] = 1
classifier_3_neg['Label'] = 0

In [221]:
x_3 = pd.concat([classifier_3_pos,classifier_3_neg], axis=0).iloc[:,0:5]
y_3 = pd.concat([classifier_3_pos,classifier_3_neg], axis=0).iloc[:,[-1]]

# Some Maths

In [222]:
def sigmoid(z):
    return 1/(1+ np.exp(-z))


In [223]:
def gradient(theta, x, y):
    
    m = x.shape[0]
    h = sigmoid(x.dot(theta.reshape(-1,1)))
    grad = (1/m)*x.T.dot(h-y)
    
    return (grad.flatten())

In [224]:
def costFunctionVectorized(theta, X, y,):
    m = X.shape[0]
    
    h = sigmoid(X.dot(theta))
    
    J = -1*(1/m)*(np.log(h).T.dot(y) + np.log(1-h).T.dot(1-y))
    
    if np.isnan(J[0]):
        return (np.inf)
    
    return (J[0])

In [235]:
res_1 = minimize(costFunctionVectorized, np.ones((x_1.shape[1],1)), args=(x_1.values, y_1.values), method="BFGS", jac=gradient)
res_2 = minimize(costFunctionVectorized, np.zeros((x_2.shape[1],1)), args=(x_2.values, y_2.values), method="BFGS", jac=gradient)
res_3 = minimize(costFunctionVectorized, np.zeros((x_3.shape[1],1)), args=(x_3.values, y_3.values), method="BFGS", jac=gradient)



In [236]:
theta_1 = res_1.x.reshape(-1,1)
theta_2 = res_2.x.reshape(-1,1)
theta_3 = res_3.x.reshape(-1,1)

In [237]:
# Mi sto inventando un dato di test
unseen_example = np.array([1, 4.8, 3.3, 1.3,0.3]).reshape(1,-1)

In [268]:
h_score_1 = sigmoid(unseen_example.dot(theta_1)).ravel()[0]
h_score_2 = sigmoid(unseen_example.dot(theta_2)).ravel()[0]
h_score_3 = sigmoid(unseen_example.dot(theta_3)).ravel()[0]

In [272]:

print('Probabilità predetta Label 0: {}\nProbabilità predetta Label 1: {}\nProbabilità predetta Label 2: {}\n'.format(h_score_1, h_score_2, h_score_3))




Probabilità predetta Label 0: 0.9999999999624847
Probabilità predetta Label 1: 0.1039837525822983
Probabilità predetta Label 2: 2.988127824706727e-26

